In [4]:
import gzip
import json
import torch

In [6]:
# Path to the dataset file
file_path = "data/red-team-attempts/red_team_attempts.jsonl.gz"

# List to store parsed JSON objects
data = []

# Open and read the compressed JSONL file
with gzip.open(file_path, "rt", encoding="utf-8") as f:
    for line in f:
        line = line.strip()  # Remove leading/trailing whitespace
        if not line:  # Skip empty lines
            continue
        try:
            json_obj = json.loads(line)  # Parse JSON
            data.append(json_obj)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {e}")  # Log errors without stopping execution

print(f"Loaded {len(data)} JSON records")

# Function to process a single JSON sample and extract numerical values
def process_sample(sample):
    """Extracts relevant numerical fields and converts them into a PyTorch tensor."""
    values = [
        sample.get("min_harmlessness_score_transcript", 0.0),  # AI harmlessness score
        sample.get("num_params", 0.0),  # Number of parameters in the language model
        sample.get("rating", 0.0),  # Red team attack success rating
        sample.get("task_description_harmlessness_score", 0.0),  # Harmlessness score of the task description
    ]
    # Ensure all values are numerical
    if all(isinstance(v, (int, float)) for v in values):
        return torch.tensor(values, dtype=torch.float32)
    return None  # Return None for invalid samples

# Process all valid samples
tensor_list = [process_sample(sample) for sample in data if isinstance(sample, dict)]
tensor_list = [t for t in tensor_list if t is not None]  # Remove None values

# Ensure tensor_list is not empty before stacking
if tensor_list:
    tensor_data = torch.stack(tensor_list)
    print("Tensor shape:", tensor_data.shape)
    print("First 3 samples:\n", tensor_data[:3])
else:
    print("No valid numerical data found in the dataset.")

Skipping invalid JSON line: Expecting value: line 1 column 4020 (char 4019)
Skipping invalid JSON line: Extra data: line 1 column 3951 (char 3950)
Skipping invalid JSON line: Extra data: line 1 column 3838 (char 3837)
Skipping invalid JSON line: Extra data: line 1 column 3844 (char 3843)
Skipping invalid JSON line: Extra data: line 1 column 755 (char 754)
Skipping invalid JSON line: Extra data: line 1 column 1026 (char 1025)
Skipping invalid JSON line: Extra data: line 1 column 1021 (char 1020)
Skipping invalid JSON line: Extra data: line 1 column 1374 (char 1373)
Skipping invalid JSON line: Extra data: line 1 column 918 (char 917)
Skipping invalid JSON line: Extra data: line 1 column 1054 (char 1053)
Skipping invalid JSON line: Extra data: line 1 column 857 (char 856)
Skipping invalid JSON line: Extra data: line 1 column 1008 (char 1007)
Skipping invalid JSON line: Extra data: line 1 column 725 (char 724)
Skipping invalid JSON line: Extra data: line 1 column 765 (char 764)
Skipping in

In [2]:
noise_file = torch.load('./data/red-team-attempts/red_team_attempts.jsonl.gz')
clean_label = noise_file['clean_label']
worst_label = noise_file['worse_label']
aggre_label = noise_file['aggre_label']
random_label1 = noise_file['random_label1']
random_label2 = noise_file['random_label2']
random_label3 = noise_file['random_label3']

UnpicklingError: Weights only load failed. In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
Please file an issue with the following so that we can make `weights_only=True` compatible with your use case: WeightsUnpickler error: Unsupported operand 31

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch.org/docs/stable/generated/torch.load.html.